In [11]:
import tensorflow as tf
from transformers import TFGPT2Model, GPT2Tokenizer
model_file_path = "C:/Users/Michael/Documents/Bangkit/Project Capstone/gpt2_saved_model.h5"
#Use Custom Objects if there is transfer learning
model = tf.keras.models.load_model(model_file_path,
                                      custom_objects={
                                          "TFGPT2Model":TFGPT2Model
                                      })


In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Set the tokenizer padder
tokenizer.add_special_tokens({"pad_token": "<pad>"})
tokenizer.padding_side = "left"

In [13]:
import re
#breakdown articles/posts into sentence
def post_to_sentences(posts):
    sentence_list = []
    for post in posts:
        post = str(post)  # Convert paragraph to string
        # Split text on whitespace after period or question mark,
        # unless preceeded by single letter or followed by word character
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', post)
        for sentence in sentences:
            sentence_list.append(sentence) 
    return sentence_list

In [18]:
import numpy as np
#BUTUH PERBAIKAN
#test inference
example = "Cooking is a creative and rewarding activity that can bring joy and satisfaction to yourself and others."
#TODO:preprocess data, predict, then interpret
example = post_to_sentences([example])
print(example)
tokenized_example = tokenizer(example, padding=True, truncation=True, max_length=100, return_tensors="tf")
print(tokenized_example)
#make prediction
example_ids = tokenized_example.input_ids
example_attention_mask = tokenized_example.attention_mask
predicted_probs = model([example_ids,example_attention_mask]) # pass both inputs here
predicted_probs = predicted_probs.numpy()
#logits = predicted_label.logits
#predicted_label = tf.argmax(logits,axis=-1)
# Map the predicted label index to the actual label
#BUTUH PERBAIKAN, INI RASANYA ADA YANG SALAH
#urutannya bener books cinema cooking gaming sport? soalnya kalo diacak predicted probnya tetep sama
label_mapping = {0: 'Gaming', 1: 'Sports', 2: 'Cinema', 3: 'Books', 4: 'Cooking'}
index = np.argmax(predicted_probs)
predicted_label = label_mapping[index]

#Print the predicted label
print("Predicted label:", predicted_label)
print("Predicted prob: ", predicted_probs)

['Cooking is a creative and rewarding activity that can bring joy and satisfaction to yourself and others.']
{'input_ids': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=
array([[28937,   278,   318,   257,  7325,   290, 23404,  3842,   326,
          460,  2222,  8716,   290, 14676,   284,  3511,   290,  1854,
           13]])>, 'attention_mask': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}
Predicted label: Gaming
Predicted prob:  [[9.8221898e-01 1.9261701e-05 1.7348541e-02 5.0486551e-06 4.0807697e-04]]
